In [1]:
import pandas as pd
import re
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
df = pd.read_csv('SEC_lit_releases_2009.csv')

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
df.head()

,Release No.,Date,Action
0,LR-21357,"Dec. 31, 2009","UTStarcom, Inc. Other Release No.: AAER-3093 See also: SEC Complaint"
1,LR-21356,"Dec. 30, 2009",Walter A. Forbes et al. Other Release No.: AAER-3091
2,LR-21355,"Dec. 29, 2009",Harold H. Jaschke See also: SEC Complaint
3,LR-21354,"Dec. 29, 2009","HomePals, LLC, HomePals Investment Club, LLC, Ronnie Eugene Bass, Jr., Abner Alabre and Brian J. Taglieri"
4,LR-21353,"Dec. 24, 2009","Homestead Properties, L.P.; Homestead Limited, L.L.C.; California Wealth Management Group, d.b.a. IFC Advisory; Heath M. Biddlecome; William C. Tak, Defendants; and Homestead Northland MHC, LLC and Jackson MHC, L.L.C., Relief Defendants"


In [5]:
# make a new column of lowercase release numbers to generate URL column
df['release_no'] = df['Release No.'].str.replace('-', '')
df['release_no'] = df['release_no'].str.lower()

In [6]:
# add the url column based on the url pattern
df['url'] = "https://www.sec.gov/litigation/litreleases/2009/" + df.release_no + ".htm"

In [7]:
# drop the release_no column
df = df.drop(['release_no'], axis=1)

In [8]:
df

,Release No.,Date,Action,url
0,LR-21357,"Dec. 31, 2009","UTStarcom, Inc. Other Release No.: AAER-3093 See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2009/lr21357.htm
1,LR-21356,"Dec. 30, 2009",Walter A. Forbes et al. Other Release No.: AAER-3091,https://www.sec.gov/litigation/litreleases/2009/lr21356.htm
2,LR-21355,"Dec. 29, 2009",Harold H. Jaschke See also: SEC Complaint,https://www.sec.gov/litigation/litreleases/2009/lr21355.htm
3,LR-21354,"Dec. 29, 2009","HomePals, LLC, HomePals Investment Club, LLC, Ronnie Eugene Bass, Jr., Abner Alabre and Brian J. Taglieri",https://www.sec.gov/litigation/litreleases/2009/lr21354.htm
4,LR-21353,"Dec. 24, 2009","Homestead Properties, L.P.; Homestead Limited, L.L.C.; California Wealth Management Group, d.b.a. IFC Advisory; Heath M. Biddlecome; William C. Tak, Defendants; and Homestead Northland MHC, LLC and Jackson MHC, L.L.C., Relief Defendants",https://www.sec.gov/litigation/litreleases/2009/lr21353.htm
5,LR-21352,"Dec. 23, 2009","Crossroads Financial Planning, Inc., and Julie M. Jarvis",https://www.sec.gov/litigation/litreleases/2009/lr21352.htm
6,LR-21351,"Dec. 23, 2009","Golden Apple Oil and Gas, Inc., et al. See also: Judgment as to Jay Budd",https://www.sec.gov/litigation/litreleases/2009/lr21351.htm
7,LR-21350,"Dec. 23, 2009","Richard E. McDonald, et al. Other Release No.: AAER-3089 See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2009/lr21350.htm
8,LR-21349,"Dec. 23, 2009","Alliance Transcription Services, Inc.",https://www.sec.gov/litigation/litreleases/2009/lr21349.htm
9,LR-21348,"Dec. 23, 2009","Rockwell Energy of Texas, LLC, Rockwell Energy Management, LLC, Gregory S. Shindler, Bradley M. James, W. Todd Smith, Stuart E. Rawitt, and Brian W. Walsh See also: SEC Complaint",https://www.sec.gov/litigation/litreleases/2009/lr21348.htm


In [9]:
# using BeautifulSoup to loop through each URL and pull all the text

def get_release_info(row):
    
    try:
        request = requests.get(row['url'])
        soup = BeautifulSoup(request.text, 'html.parser')
        
        full_text = soup.findAll(text=True)
        
    except:
        return pd.Series({})

    return pd.Series({
        'full_text': full_text,
    })

In [10]:
df_litreleases_2009 = df.apply(get_release_info, axis=1).join(df)

In [11]:
df_litreleases_2009.head()

full_text  \
0  [HTML PUBLIC "-//W3C//DTD HTML 4.0 Transitional//EN", 
, 
, 
, UTStarcom, Inc.: Lit. Rel. No. 21357 / December 31, 2009, 
, 
,  BEGIN HEADER , 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, Home,  | , Previous Page, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
, 
,  END HEADER , 
,  Begin text , 
, U.S. SECURITIES AND EXCHANGE COMMISSION, 
, Litigation Release No. 21357 / December 31, 2009, 
, Accounting and Auditing Enforcement Release No. 3093 / December 31, 2009, 
, Securities and Exchange Commission v. UTStarcom, Inc., , Case No. CV-09-6094 (JSW) (N.D. Cal. filed Dec. 31, 2009), 
, SEC CHARGES CALIFORNIA TELECOM COMPANY WITH BRIBERY AND OTHER FCPA VIOLATIONS, 
, The Securities and Exchange Commission today charged Alameda, Calif.-based telecommunications company UTStarcom, Inc. with violations of the Foreign Corrupt Practices Act (FCPA) for authorizing unlawful payments to foreign government officials in Asia., 
, UTStarcom agreed to settle the SEC's charges and pay a $1.5 million penalty among other remedies. In a related criminal case, the U.S. Department of Justice announced today that UTStarcom agreed to pay an additional $1.5 million fine., 
, The SEC's complaint, filed in the U.S. District Court for the Northern District of California, alleges that UTStarcom's wholly-owned subsidiary in China paid nearly $7 million between 2002 and 2007 for hundreds of overseas trips by employees of Chinese government-controlled telecommunications companies that were customers of UTStarcom, purportedly to provide customer training. In reality, the trips were entirely or primarily for sightseeing., 
, The SEC further alleges that UTStarcom provided lavish gifts and all-expenses paid executive training programs in the U.S. for existing and potential foreign government customers in China and Thailand. UTStarcom also purported to hire individuals affiliated with foreign government customers to work in the U.S. and provided them with work visas, when in reality the individuals did no work for UTStarcom. According to the SEC, UTStarcom also made improper payments to sham consultants in China and Mongolia while knowing that they would pay bribes to foreign government officials., 
, The SEC's complaint charges UTStarcom with violations of the anti-bribery, books and records, and internal controls provisions of the FCPA (Sections 30A, 13(b)(2)(A) and 13(b)(2)(B) of the Securities Exchange Act of 1934, respectively). UTStarcom agreed, without admitting or denying the charges, to the entry of a permanent injunction against FCPA violations and to provide the SEC with annual FCPA compliance reports and certifications for four years, in addition to paying the $1.5 million penalty., 
, The SEC acknowledges the assistance of the Department of Justice during the investigation., 
, Complaint, 
, 
, SEC Complaint, 
, 
 
, 
, http://www.sec.gov/litigation/litreleases/2009/lr21357.htm, 
,  BEGIN FOOTER , 
, 
, 
, 
, Home,  | , Previous Page, 
, ...]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [12]:
df_litreleases_2009.dtypes

full_text      object
Release No.    object
Date           object
Action         object
url            object
dtype: object

In [13]:
df_litreleases_2009['Respondents'] = df_litreleases_2009['Action']

In [14]:
# rearrange columns
df_litreleases_2009 = df_litreleases_2009[['Release No.','Respondents','Date','url', 'full_text']]
df_litreleases_2009.head()

Release No.  \
0  LR-21357     
1  LR-21356     
2  LR-21355     
3  LR-21354     
4  LR-21353     

                                                                                                                                                                                                                                    Respondents  \
0  UTStarcom, Inc. Other Release No.: AAER-3093 See also: SEC Complaint                                                                                                                                                                           
1  Walter A. Forbes et al. Other Release No.: AAER-3091                                                                                                                                                                                           
2  Harold H. Jaschke See also: SEC Complaint                                                                                                                                                                                                      
3  HomePals, LLC, HomePals Investment Club, LLC, Ronnie Eugene Bass, Jr., Abner Alabre and Brian J. Taglieri                                                                                                                                      
4  Homestead Properties, L.P.; Homestead Limited, L.L.C.; California Wealth Management Group, d.b.a. IFC Advisory; Heath M. Biddlecome; William C. Tak, Defendants; and Homestead Northland MHC, LLC and Jackson MHC, L.L.C., Relief Defendants   

            Date                                                          url  \
0  Dec. 31, 2009  https://www.sec.gov/litigation/litreleases/2009/lr21357.htm   
1  Dec. 30, 2009  https://www.sec.gov/litigation/litreleases/2009/lr21356.htm   
2  Dec. 29, 2009  https://www.sec.gov/litigation/litreleases/2009/lr21355.htm   
3  Dec. 29, 2009  https://www.sec.gov/litigation/litreleases/2009/lr21354.htm   
4  Dec. 24, 2009  https://www.sec.gov/litigation/litreleases/2009/lr21353.htm   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [15]:
# Save to CSV
df_litreleases_2009.to_csv("litreleases_2009_scraped.csv", index=False)